In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation

from simulation.loader import load_environment

In [ ]:
def update_density(density: np.ndarray, r: float = 0.5, e: float = 0.95):
    """
    Parameters
    ----------
    density : np.ndarray
    r : float, [0, 1]
        Diffusion parameter. For a tile `r` parts remain on the tile and `(1-r)` parts evenly distribute on the
        neighboring tiles.
    e : float, [0, 1]
        Runoff parameter. The density is multiplied by `e`.
    """
    next_density = np.zeros_like(density, dtype=float)

    next_density += r * density
    next_density[1:,:] += (r / 4) * density[:-1,:]
    next_density[:-1,:] += (r / 4) * density[1:,:]
    next_density[:,1:] += (r / 4) * density[:,:-1]
    next_density[:,:-1] += (r / 4) * density[:,1:]

    return next_density * e

In [ ]:
env, *_ = load_environment("../data/test-map-1.txt", 1)
density = np.zeros_like(env)

density[20, 30] = 1
density[50, 20] = 1
density[10, 10] = 1
density[30, 0] = 1

fig, ax = plt.subplots()
particle_layer = ax.imshow(density.T)


def update(frame):
    global density
    density = update_density(density)

    pos1 = (20 + frame, 30 - frame)
    pos2 = (50 - frame, 20)
    density[pos1] += 1
    density[pos2] += 1
    density[10, 10] += 1
    density[30, 0] += 1

    particle_layer.set_data(density.T)
    return particle_layer,

animation = FuncAnimation(fig, update, frames=20)
animation.save("diffusion.mp4", fps=10, dpi=300)
